In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(f'{os.getcwd()}/../')

In [3]:
from typing import Union, Optional, Any, Set, Sequence
from abc import ABC

import functools
import random

import gymnasium
import numpy as np
import numpy.typing as npt
import gymnasium.spaces
from gymnasium.spaces import Discrete, Space, Dict

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers
from pettingzoo.test import api_test
from pettingzoo.utils.env import AECIterable, AgentID, ActionType, ObsType

from src.wrapper import RestrictionWrapper

In [4]:
ROCK = 0
PAPER = 1
SCISSORS = 2
NONE = 3
MOVES = ["ROCK", "PAPER", "SCISSORS", "None"]
NUM_ITERS = 100
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}

class RPSEnvironment(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """
        self.possible_agents = ["player_" + str(r) for r in range(2)]

        # optional: a mapping between agent name and ID
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # optional: we can define the observation and action spaces here as attributes to be used in their corresponding methods
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Discrete(4) for agent in self.possible_agents
        }
        self.render_mode = render_mode

    # Observation space should be defined here.
    # lru_cache allows observation and action spaces to be memoized, reducing clock cycles required to get each agent's space.
    # If your spaces change over time, remove this line (disable caching).
    # @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        # return Discrete(4)
        return self._observation_spaces[agent]

    # Action space should be defined here.
    # If your spaces change over time, remove this line (disable caching).
    # @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        # return Discrete(3)
        return self._action_spaces[agent]

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return

        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        # observation of one agent is the previous state of the other
        return np.array(self.observations[agent])

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: NONE for agent in self.agents}
        self.observations = {agent: NONE for agent in self.agents}
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            return

        agent = self.agent_selection

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        self._cumulative_rewards[agent] = 0

        # stores action of current agent
        self.state[self.agent_selection] = action

        # collect reward if it is the last agent to act
        if self._agent_selector.is_last():
            # rewards for all agents are placed in the .rewards dictionary
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = REWARD_MAP[
                (self.state[self.agents[0]], self.state[self.agents[1]])
            ]

            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= NUM_ITERS for agent in self.agents
            }

            # observe the current state
            for i in self.agents:
                self.observations[i] = self.state[
                    self.agents[1 - self.agent_name_mapping[i]]
                ]
        else:
            # necessary so that observe() returns a reasonable observation at all times.
            self.state[self.agents[1 - self.agent_name_mapping[agent]]] = NONE
            # no rewards are allocated until both players give an action
            self._clear_rewards()

        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        if self.render_mode == "human":
            self.render()

In [34]:
class Restriction(ABC, gymnasium.Space):
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}'

class DiscreteRestriction(Restriction, ABC):
    pass

class DiscreteSetRestriction(DiscreteRestriction):
    def __init__(self, *, allowed_actions: Optional[Set[int]] = None, base_space: Optional[Discrete] = None):
        if (allowed_actions is None) == (base_space is None):
            raise ValueError

        self.allowed_actions = allowed_actions if allowed_actions is not None else set(range(base_space.start, base_space.start + base_space.n))

    @property
    def is_np_flattenable(self) -> bool:
        return True
    
    def sample(self) -> int:
        return random.choice(tuple(self.allowed_actions))

    def contains(self, x: int) -> bool:
        return x in self.allowed_actions
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.allowed_actions})'

class DiscreteVectorRestriction(DiscreteRestriction):
    def __init__(self, allowed_actions: np.ndarray[bool], start: int = 0):
        self.allowed_actions = allowed_actions
        self.start = start

    @property
    def is_np_flattenable(self) -> bool:
        return True
    
    def sample(self) -> int:
        return self.start + random.choice(tuple(index for index, value in enumerate(self.allowed_actions) if value))

    def contains(self, x: int) -> bool:
        return self.allowed_actions[x - self.start]
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.allowed_actions})'


In [38]:
class Restrictor:
    default_restriction_classes = {
        gymnasium.spaces.Discrete: DiscreteSetRestriction,
        gymnasium.spaces.Box: DiscreteVectorRestriction
    }

    def __init__(self, env, restriction_classes=None) -> None:
        self.action_spaces = {agent: env.action_space(agent) for agent in env.possible_agents}
        self.restriction_classes = restriction_classes or self.default_restriction_classes

    def act(self, observation):
        print(f'act: {observation=}')
        state, agent, observation = observation

        return self.restriction_classes[type(self.action_spaces[agent])](base_space=self.action_spaces[agent])

In [39]:
class RestrictorActionSpace(ABC, gymnasium.Space):
    @property
    def is_np_flattenable(self) -> bool:
        return False

    def sample(self) -> Restriction:
        return DiscreteVectorRestriction(np.array([True]))
        
    def contains(self, x: Restriction) -> bool:
        return True
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}'

In [40]:
def create_policies(env):
    return { agent: lambda _: env.action_space(agent).sample() for agent in env.possible_agents }

In [41]:
env = RPSEnvironment()
wrapper = RestrictionWrapper(env, Discrete(1), RestrictorActionSpace(), preprocess_restrictor_observation_fn=lambda env, agent, observations: (env.state, agent, observations[agent]))
policies = create_policies(env)

policies['restrictor_0'] = Restrictor(env).act

wrapper.reset()
for agent in wrapper.agent_iter(max_iter=10):
    observation, reward, termination, truncation, info = wrapper.last()
    print(f'{agent=}, {observation=}, {reward=}')
    action = policies[agent](observation)
    print(f'{action=}')
    wrapper.step(action)

agent='restrictor_0', observation=({'player_0': 3, 'player_1': 3}, 'player_0', None), reward=0.0
act: observation=({'player_0': 3, 'player_1': 3}, 'player_0', None)
action=DiscreteSetRestriction({0, 1, 2})
agent='player_0', observation={'observation': array(3), 'restriction': DiscreteSetRestriction({0, 1, 2})}, reward=0
action=2
self.terminations={'player_0': False, 'player_1': False, 'restrictor_0': False}, self.truncations={'player_0': False, 'player_1': False, 'restrictor_0': False}
agent='restrictor_0', observation=({'player_0': 2, 'player_1': 3}, 'player_1', None), reward=0.0
act: observation=({'player_0': 2, 'player_1': 3}, 'player_1', None)
action=DiscreteSetRestriction({0, 1, 2})
agent='player_1', observation={'observation': array(3), 'restriction': DiscreteSetRestriction({0, 1, 2})}, reward=0
action=0
self.terminations={'player_0': False, 'player_1': False, 'restrictor_0': False}, self.truncations={'player_0': False, 'player_1': False}
agent='restrictor_0', observation=({'play